In [34]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
from torch.utils.tensorboard import SummaryWriter
from Method_timer import Method_timer

In [28]:
@Method_timer
def train(net, epochs, trainloader, optimizer, criterion=nn.CrossEntropyLoss(), title="", writer=None):
    print(
        f"training {title} network for {epochs} epochs, {'tensorboard enabled' if writer else 'no tensorboard enabled'}")
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    
    for epoch in range(epochs):
        running_loss = 0.0
        running_correct = 0.0
        for i, data in enumerate(trainloader, 0):
            
            # get the inputs; data is a list of [inputs, labels]
            inputs, labels = data[0].to(device), data[1].to(device)

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            # print statistics
            running_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            running_correct += (predicted == labels).sum().item()
            
            if i % 2000 == 1999:    # print every 2000 mini-batches
                print('[%d, %5d] loss: %.3f' %
                        (epoch + 1, i + 1, running_loss / 2000))
                if writer:
                    writer.add_scalar(
                        'training loss, ' + title, running_loss/2000, epoch)
                    writer.add_scalar('accuracy, '+title,
                                        running_correct/2000, epoch)

                running_loss = 0.0
                running_correct = 0.0

    print('Finished Training')

In [23]:
def validate(net, testloader):
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    print("testing network:")
    correct = 0
    total = 0
    
    with torch.no_grad():
        for data in testloader:
            inputs, labels = data[0].to(device), data[1].to(device)
            outputs = net(inputs)
            
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        print("total accuracy of net: %.2f%%" % (correct/total*100))

In [29]:
# Transform

transform_train = transforms.Compose([
    transforms.Resize(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

In [35]:
batch_size = 32

trainset = torchvision.datasets.CIFAR10(root='./data', 
                                        train=True,
                                        download=True, 
                                        transform=transform_train)               

trainloader = torch.utils.data.DataLoader(trainset, 
                                          batch_size=batch_size,
                                          shuffle=True)

testset = torchvision.datasets.CIFAR10(root='./data', 
                                       train=False,
                                       download=True, 
                                       transform=transform_test)

testloader = torch.utils.data.DataLoader(testset, 
                                         batch_size=batch_size,
                                         shuffle=False)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 
           'dog', 'frog', 'horse', 'ship', 'truck')

#writer = SummaryWriter()

Files already downloaded and verified
Files already downloaded and verified


In [36]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# ==== Alex Model ====

alexnet_pretrained = torchvision.models.alexnet(pretrained=True) 
example, _ = data.next()
    
alexnet_pretrained.eval()


NameError: name 'data' is not defined

In [22]:
# Add an extra outputlayer with 10 outputs
# Not sure how to add extra layer
# Instead change the last layer to 10 outputs

alexnet_pretrained.classifier[6] = nn.Linear(4096,10)
alexnet_pretrained.eval()

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [39]:
# === FineTunning ===

alexnet_pretrained.to(device)
alexnet_finetuning = alexnet_pretrained

train(alexnet_finetuning,
      epochs = 1,
      trainloader = trainloader,
      optimizer = optim.Adam(alexnet_finetuning.parameters(), lr = 0.001),
      criterion = nn.CrossEntropyLoss(),
      title = "Fine Tuning AlexNet on CIFAR-10")

TypeError: timer() got an unexpected keyword argument 'epochs'

In [26]:
# Validate
validate(alexnet_finetuning, testloader)

testing network:
total accuracy of net: 10.00%


In [28]:
# === FineTunning ===

alexnet_pretrained.to(device)
alexnet_finetuning = alexnet_pretrained

train(alexnet_finetuning,
      epochs = 5,
      trainloader = trainloader,
      optimizer = optim.SGD(alexnet_finetuning.parameters(), lr=0.001, momentum=0.9),
      criterion = nn.CrossEntropyLoss(),
      title = "Fine Tuning AlexNet on CIFAR-10")

training Fine Tuning AlexNet on CIFAR-10 network for 5 epochs, no tensorboard enabled
[1,   200] loss: 2.302
[1,   400] loss: 2.303
[1,   600] loss: 2.303
[1,   800] loss: 2.303
[1,  1000] loss: 2.303
[1,  1200] loss: 2.303
[1,  1400] loss: 2.303
[2,   200] loss: 2.303
[2,   400] loss: 2.303
[2,   600] loss: 2.303
[2,   800] loss: 2.303
[2,  1000] loss: 2.303
[2,  1200] loss: 2.303
[2,  1400] loss: 2.303
[3,   200] loss: 2.303
[3,   400] loss: 2.302
[3,   600] loss: 2.303
[3,   800] loss: 2.303
[3,  1000] loss: 2.303
[3,  1200] loss: 2.303
[3,  1400] loss: 2.303
[4,   200] loss: 2.303
[4,   400] loss: 2.303
[4,   600] loss: 2.303
[4,   800] loss: 2.303
[4,  1000] loss: 2.303
[4,  1200] loss: 2.303
[4,  1400] loss: 2.303
[5,   200] loss: 2.303
[5,   400] loss: 2.303
[5,   600] loss: 2.303
[5,   800] loss: 2.303
[5,  1000] loss: 2.303
[5,  1200] loss: 2.303
[5,  1400] loss: 2.303
Finished Training


In [29]:
# Validate
validate(alexnet_finetuning, testloader)

testing network:
total accuracy of net: 10.00%


In [9]:
transform = transforms.Compose([
            transforms.Scale(256),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ])

batch_size = 4

trainset = torchvision.datasets.CIFAR10(root='./data', 
                                        train=True,
                                        download=True, 
                                        transform=transform)               

trainloader = torch.utils.data.DataLoader(trainset, 
                                          batch_size=batch_size,
                                          shuffle=True)

testset = torchvision.datasets.CIFAR10(root='./data', 
                                       train=False,
                                       download=True, 
                                       transform=transform)

testloader = torch.utils.data.DataLoader(testset, 
                                         batch_size=batch_size,
                                         shuffle=False)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 
           'dog', 'frog', 'horse', 'ship', 'truck')

/usr/local/lib/python3.7/dist-packages/torchvision/transforms/transforms.py:285: UserWarning: The use of the transforms.Scale transform is deprecated, please use transforms.Resize instead.
  warnings.warn("The use of the transforms.Scale transform is deprecated, " +


Files already downloaded and verified
Files already downloaded and verified


In [10]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
alexnet = torchvision.models.alexnet(pretrained=True) 
alexnet.classifier[6] = nn.Linear(4096,10)
alexnet.eval()
alexnet.to(device)

train(net = alexnet,
      epochs = 5,
      trainloader = trainloader,
      optimizer = optim.SGD(alexnet.parameters(), lr=0.001, momentum=0.9),
      criterion = nn.CrossEntropyLoss()
      )


training  network for 5 epochs, no tensorboard enabled
[1,  2000] loss: 1.266
[1,  4000] loss: 0.911
[1,  6000] loss: 0.791
[1,  8000] loss: 0.711
[1, 10000] loss: 0.678
[1, 12000] loss: 0.623
[2,  2000] loss: 0.478
[2,  4000] loss: 0.478
[2,  6000] loss: 0.492
[2,  8000] loss: 0.485
[2, 10000] loss: 0.487
[2, 12000] loss: 0.470
[3,  2000] loss: 0.321
[3,  4000] loss: 0.312
[3,  6000] loss: 0.334
[3,  8000] loss: 0.344
[3, 10000] loss: 0.325
[3, 12000] loss: 0.344
[4,  2000] loss: 0.211
[4,  4000] loss: 0.209
[4,  6000] loss: 0.236
[4,  8000] loss: 0.243
[4, 10000] loss: 0.249
[4, 12000] loss: 0.257
[5,  2000] loss: 0.143
[5,  4000] loss: 0.163
[5,  6000] loss: 0.160
[5,  8000] loss: 0.201
[5, 10000] loss: 0.183
[5, 12000] loss: 0.211
Finished Training


In [11]:
# Validate
validate(alexnet, testloader)

testing network:
total accuracy of net: 83.77%


In [14]:
alexnet2 = torchvision.models.alexnet(pretrained=True) 
for param in alexnet2.parameters():
    param.requires_grad = False
    
alexnet2.classifier[6] = nn.Linear(4096, 10)
alexnet2.to(device)

train(net = alexnet2,
      epochs = 5,
      trainloader = trainloader,
      optimizer = optim.SGD(alexnet2.parameters(), lr=0.001, momentum=0.9),
      criterion = nn.CrossEntropyLoss()
      )

training  network for 5 epochs, no tensorboard enabled
[1,  2000] loss: 2.242
[1,  4000] loss: 2.167
[1,  6000] loss: 2.236
[1,  8000] loss: 2.234
[1, 10000] loss: 2.194
[1, 12000] loss: 2.230
[2,  2000] loss: 2.163
[2,  4000] loss: 1.948
[2,  6000] loss: 2.110
[2,  8000] loss: 2.160
[2, 10000] loss: 2.202
[2, 12000] loss: 2.185
[3,  2000] loss: 2.166
[3,  4000] loss: 2.097
[3,  6000] loss: 2.110
[3,  8000] loss: 2.159
[3, 10000] loss: 2.055
[3, 12000] loss: 2.296
[4,  2000] loss: 1.985
[4,  4000] loss: 2.031
[4,  6000] loss: 2.159
[4,  8000] loss: 2.112
[4, 10000] loss: 2.135
[4, 12000] loss: 2.094
[5,  2000] loss: 2.036
[5,  4000] loss: 2.042
[5,  6000] loss: 2.084
[5,  8000] loss: 2.183
[5, 10000] loss: 2.077
[5, 12000] loss: 2.043
Finished Training


In [15]:
validate(alexnet2, testloader)

testing network:
total accuracy of net: 73.90%
